<a href="https://colab.research.google.com/github/AnnSenina/Python_for_CL/blob/main/notebooks/Python_12_pymorphy%2C_mystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лемматизация, морф.анализ

Для русского основых лемматизатора два: Pymorphy и Mystem.

##PyMorphy2

Не самый лучший в смысле точности, но очень популярный и простой инструмент для лемматизации

In [ ]:
!pip install pymorphy2

In [2]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

### основная функция - pymorphy.parse
     
Она похожа на analyze в mystem, возвращает список объектов Parse

Первый в списке - самый вероятный разбор (у каждого есть score)

Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать

In [ ]:
morph.parse('человек')

Документация [здесь](https://pymorphy2.readthedocs.io/en/stable/user/guide.html#id3), обозначения для граммем [здесь](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs)

In [ ]:
# сделаем красиво
print('Cлово - ', morph.parse('человеком')[0].word)
print('Лемма слова - ', morph.parse('человеком')[0].normal_form)
print('Грамматическая информация слова - ', morph.parse('человеком')[0].tag)
print('Часть речи слова - ', morph.parse('человеком')[0].tag.POS)
print('Род слова - ', morph.parse('человеком')[0].tag.gender)
print('Число слова - ', morph.parse('человеком')[0].tag.number)
print('Падеж слова - ', morph.parse('человеком')[0].tag.case)

In [ ]:
# для лемматизации нужна именно эта команда
morph.parse('человеком')[0].normal_form

Давайте снова возьмем Сэлинджера...

Увы! pymorphy не умеет токенизировать, поэтому...

In [ ]:
# все то, что надо импортировать из NLTK
#!pip install nltk

import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import download
download('punkt')
download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian') 

In [ ]:
# собрала препроцессинг в 1 функцию - пока только до стемминга
def clean_text(text_str):
  text_str = text_str.lower() # нижний регистр
  text_list_nltk = word_tokenize(text_str) # токенизация
  stop_words = stopwords.words('russian') 
  text_without_punkt = [word for word in text_list_nltk if word[0].isalpha()] # удалить пунктуацию из списка токенов
  text_clean = [word for word in text_without_punkt if word not in stop_words] # чистим от стоп-слов
  return text_clean

with open('bananafish.txt', 'r', encoding='utf-8') as file:
  bananafish = file.read()
bananafish = bananafish.replace('\ufeff', '')
bananafish = bananafish.replace('\xa0', ' ')
bananafish_tokens = clean_text(bananafish)
bananafish_tokens

In [ ]:
words_lemmatized = []
for word in bananafish_tokens:
    result = morph.parse(word)
    most_probable_result = result[0] ## почему мы берем первый разбор? см.в этом месте: https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct
    normal_form = most_probable_result.normal_form
    words_lemmatized.append(normal_form)

words_lemmatized
# print(words_lemmatized)

In [ ]:
from collections import Counter

Counter(words_lemmatized).most_common(10)
#print(Counter(words_lemmatized).most_common(10))

In [ ]:
import nltk

freq_bigramms = Counter(nltk.bigrams(words_lemmatized))
freq_bigramms.most_common(10)
#print(Counter(words_lemmatized).most_common(10))

# что-то пошло не так?

#Более умный лемматизатор MyStem

Сравнительная точность pymorphy и mystem для русского языка [здесь](http://web-corpora.net/wsgi/mystemplus.wsgi/mystemplus/compare_table/)


In [ ]:
!pip install pymystem3

Только для Colab!

In [ ]:
!pip install pymystem3==0.1.10

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

Дальше уже для всех!

In [4]:
from pymystem3 import Mystem
mystem = Mystem()

In [ ]:
mystem = Mystem()

In [ ]:
mystem.lemmatize('mystem даже сам умеет токенизировать текст')

In [ ]:
words_lemmatized_mystem = mystem.lemmatize(' '.join(bananafish_tokens))
words_lemmatized_mystem
#print(words_lemmatized_mystem)

In [ ]:
print(''.join(words_lemmatized_mystem))

In [ ]:
from collections import Counter

Counter(words_lemmatized_mystem).most_common(10)
#print(Counter(words_lemmatized_mystem).most_common(10))

### морф. анализ в MyStem

In [ ]:
sometext = "Задача NLI важна для компьютерных лингвистов"
analyzed = mystem.analyze(sometext) 
print(analyzed) 

In [ ]:
# возвращает список словарей
# каждый словарь имеет либо одно поле 'text' (когда попался пробел) или text и analysis
# в analysis снова список словарей с вариантами разбора (первый самый вероятный)
# поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
# analysis - может быть пустым списком
print('Слово - ', analyzed[0]['text'])
print('Разбор слова - ', analyzed[0]['analysis'][0])
print('Лемма слова - ', analyzed[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', analyzed[0]['analysis'][0]['gr'])

Попробуйте сами придумать предложение с омонимами и сравнить морф.анализ через pymorphy и mystem

Сравните лемматтизацию - лемматизируйте предложение с морф. неоднозначностью

### Доп. задание

Попробуйте написать новую функцию, которая принимает на вход текст как строку и возвращает список лемматизированных слов

In [ ]:
# ваша функция


Существуют и другие библиотеки для лемматизации

русский язык:

- [RNNmorph](https://github.com/IlyaGusev/rnnmorph)
- [deeppavlov](http://docs.deeppavlov.ai/en/master/)

другие языки:
- [UralicNLP](https://github.com/mikahama/uralicNLP)
- [hfst от Apertium](https://wiki.apertium.org/wiki/Hfst)
- [Stanza](https://stanfordnlp.github.io/stanza/)
- [SpaCy](https://spacy.io/usage/linguistic-features#morphology)
- [Trankit](https://trankit.readthedocs.io/en/latest/posdep.html)